# 嘉沃资管量化研究员笔试题

注意事项：
1. 请独立完成题目,严禁使用外部插件如chatgpt【一经发现,笔试成绩作废! 】
2. 整套题目回答时间为1小时,共计7道题，每道题基础分值为10分，满分100分【部分题目根据回答情况会有加分项，但请合理安排时间！】
3. 回答问题的编程语言统一使用Python3,可以调用第三方库来解答问题
4. 笔试题目在该jupyter notebook中规定位置作答，可以在【Ans】与下一题题干之间根据自身编程习惯插入代码块及文字补充说明
5. 完成题目时允许参考书籍或网络资料，但请不要照搬代码
6. 作答完成后，通过邮件提交该notebook文件：命名格式为：提交日期+姓名.ipynb

## File Overview

This code contains the following key functional areas:

| Section | Purpose |
|----------|----------|
| [1. Utility Functions](#1-utility-functions) | Basic tools such as vector sampling, interpolation, and Bessel/Gamma/Possion draws. |
| [2. COS Expansion Methods](#2-cos-expansion-methods) | Computes Fourier-Cosine coefficients and characteristic functions for option pricing. |
| [3. Conditional Variance Simulation](#3-conditional-variance-simulation) | Generates variance paths under the CTCM process. |
| [4. Heston Characteristic Functions](#4-heston-characteristic-functions) | Analytical solutions for characteristic functions in Heston-type models. |
| [5. Composite Spot Dynamics](#5-composite-spot-dynamics) | Combines jump-diffusion (JD) and Heston volatility components to approximate VIX process. |
| [6. Pricing Formula](#6-pricing-formula) | Core simulation and valuation engine for VIX futures and options. |
| [7. Error Evaluation](#7-error-evaluation) | Calibration objective functions for optimization. |

---

## 1. Utility Functions

### `bisect_left(const VectorXd& a, double x)`
Performs a **binary search** to find the index of the first element in ascending vector `a` that is not less than `x`.  
Equivalent to Python's `bisect_left`.

**Usage:** Used in sampling and numerical interpolation.

---

### `calculate_x_simu(const VectorXd& rand, const MatrixXd& F, const MatrixXd& x)`
Generates simulated samples from a given cumulative distribution `F`.  
Each random number `rand(i)` is mapped to the corresponding segment in `F` and linearly interpolated to produce the simulated variable.

**Inputs:**
- `rand`: random uniform samples `[0,1]`
- `F`: cumulative probability matrix
- `x`: quantile grid

**Output:** A `VectorXd` of simulated values.

---

### `besseli_ratio(double v, double z)`  
Computes the ratio $$ I_{v+1}(z) / I_v(z) $$ of modified Bessel functions, required for CIR or Bessel-type processes.

---

### `sample_bessel(double v, double z)`  
Samples from a **Bessel distribution** using the above ratio and normal approximation.  
Used to generate paths of conditional variance under CTCM or CIR processes.

---

## 2. COS Expansion Methods

### `calculate_Vk_spx(double L, double A, int N)`
Computes the **Fourier-Cosine coefficients $V_k$** for a given payoff function, required by the COS pricing method.

### `calculate_chi(double k, double a, double b)`
Computes the cosine integral part of the Fourier projection.

### `calculate_psi(double k, double a, double b)`
Computes the sine integral part of the Fourier projection.

---

## 3. Conditional Variance Simulation

### `get_quants(const VectorXd& para, double T)`
Derives **quantitative parameters** for the conditional distribution of volatility under a **CTCM** or **CIR** process.

**Outputs:**
- `quant`: vector of quantile levels,
- `gamma_n`, `lambda_n`: shape and non-centrality parameters for chi-square approximation.

---

### `get_conditional_V(tuple<VectorXd, VectorXd, VectorXd> quants, double v0, const VectorXd& v_T, double T)`
Generates the **conditional volatility path  $V_T$ ** given the initial variance  $v_0$  and terminal variance  $v_T$ , using a Gamma–Poisson–Bessel decomposition.

---

### `get_X1`, `get_X2`, `get_X3`
Helper components for `get_conditional_V`, producing:
- `X1`: Poisson-Gamma mixture term,
- `X2`: central Gamma term,
- `X3`: Bessel correction term.

---

## 4. Heston Characteristic Functions

### `char_func_Heston(const VectorXcd& u, const VectorXd& para, const VectorXd& v0, double T)`
Computes the **Heston model characteristic function** for a complex vector `u`.

$$
\phi(u) = \exp(A(u,T) + B(u,T)v_0)
$$
Used for Fourier-based pricing and VIX dynamics.

---

### `char_func_Heston_number(const complex<double>& u, const VectorXd& para, const VectorXd& v0, double T)`
Same as above but for a single complex input  $u$ .  
Used in analytical components like `spot_Composite_JD`.

---

## 5. Composite Spot Dynamics

### `spot_Composite_JD(const VectorXd& para, const VectorXd& v)`
Builds the **jump-diffusion (JD) component** of the spot variance model, capturing the effect of CGMY-type jumps.

**Outputs:**
- $a(v), b, c(v)$ : coefficients of linearized VIX approximation
$$
VIX = a(v) \cdot u_T + b \cdot v_T + c(v)
$$

---

### `spot_Composite_Heston(const VectorXd& para, const VectorXd& v)`
Analogous to `spot_Composite_JD`, but for the **pure Heston** stochastic volatility component.

---

## 6. Pricing Formula

### `pricing_CTCM_JD(const list<VectorXd>& TK_V, const VectorXd& vol_VIX, const VectorXd& futures, const VectorXd& para)`
The **core simulation and pricing routine**.

#### Key Steps:
1. Loop over maturities `t` in `TK_V`.
2. For each:
   - Compute variance parameters via `get_quants()`.
   - Sample variance paths `v_T`, `V_T`, and auxiliary processes `u_T`, `u2_T`.
   - Compute coefficients using `spot_Composite_JD` and `spot_Composite_Heston`.
   - Combine both processes:
    $$
     VIX = (a u_T + b v_T + c) + (a_2 u_{2T} + b_2 v_T + c_2)
    $$
   - Derive VIX futures price:
     $$
     F = 100 \times \mathbb{E}[\sqrt{VIX}]
    $$
   - Compute option prices via Monte Carlo simulation:
    $$
     C(K) = \mathbb{E}[\max(VIX / F - K/F, 0)]
    $$
   - Infer implied volatility via `implied_volatility`.

#### Outputs:
- `vol_surf`: modeled implied volatility surface of VIX options.
- `VIX_model`: option prices.
- `mean_fut_error`: mean absolute deviation between model and futures data.

---

## 7. Error Evaluation

### `error_function(...)`
Computes the **scalar calibration loss function** used to fit parameters to observed market data.

**Procedure:**
1. Construct full parameter vector.
2. Call `spx_options_pricing_formula_cos_CTCL(...)`.
3. Compute RMSE and relative errors for both SPX and VIX implied volatilities.

**Output:** single scalar objective value (used in optimization).

---

### `error_function2(...)`
Variant of `error_function` returning **a detailed vector of calibration diagnostics**, including:
- errors for SPX/VIX,
- absolute, relative, and mean errors,
- calibration date,
- parameter values.

Used for logging and convergence analysis.

---

## ⚙️ Dependencies

- **Eigen**: matrix and vector operations  
- **Boost::math**: root solving and special functions  
- **Boost::random**: non-central chi-squared, Poisson, Gamma distributions  
- **csv2**: CSV file reading for external precomputed simulation data  
- **C++17 STL**: tuple, list, random engines, complex numbers  

---

## 🧩 Typical Workflow

1. **Parameter Initialization:** set `para` vector (model parameters).
2. **Data Loading:** read option maturities `TK_S`, `TK_V`, and implied vol surfaces.
3. **Run Pricing:**  
   ```cpp
   auto [vol_surf, VIX_model, fut_error] = pricing_CTCM_JD(TK_V, vol_VIX, futures, para);
   

# Financial Mathematics Library - Complete Documentation

## Table of Contents
1. [Core Functionality Overview](#core-functionality-overview)
2. [Mathematical Models](#mathematical-models)
3. [Numerical Methods](#numerical-methods)
4. [Option Pricing Framework](#option-pricing-framework)
5. [Characteristic Functions](#characteristic-functions)
6. [Simulation Methods](#simulation-methods)
7. [Utility Functions](#utility-functions)
8. [Error Handling & Calibration](#error-handling--calibration)
9. [Dependencies](#dependencies)
10. [Usage Examples](#usage-examples)

## Core Functionality Overview

This C++ library provides a comprehensive implementation of advanced financial mathematics models, specializing in:

- Stochastic volatility modeling
- Jump-diffusion processes
- Fourier-based option pricing
- Monte Carlo simulations
- Implied volatility calculations

The code leverages Eigen for high-performance linear algebra and Boost for advanced mathematical functions.

## Mathematical Models

### CGMY Model Components
cpp
VectorXcd char_CGMY(const VectorXcd& u, double C, double G, double M, double Y)

- Implements the characteristic function for the CGMY process
- Parameters: Complex vector `u`, CGMY parameters (C, G, M, Y)
- Returns: Characteristic function values

cpp
VectorXcd char_CGMY_neg(const VectorXcd& u, double C, double G, double Y)

- Characteristic function for negative jumps only in CGMY model

### Heston Model Components
cpp
VectorXcd char_func_Heston(const VectorXcd& u, const VectorXd& para, const VectorXd& v0, double T)

- Computes Heston model characteristic function
- Parameters: Complex vector `u`, model parameters, initial variance `v0`, maturity `T`

## Numerical Methods

### Runge-Kutta Integrators
cpp
void rk4(...) 
void rk4_T(...)
void rk4_2(...)

- Multiple implementations of 4th-order Runge-Kutta method
- Specialized versions for:
  - General ODEs
  - Time-dependent characteristic functions
  - Financial applications

### Root Finding & Interpolation
cpp
int bisect_left(const Ref<const VectorXd>& a, const double& x)

- Binary search implementation (bisect_left equivalent)
- Used for efficient interpolation in pricing formulas

## Option Pricing Framework

### SPX Options Pricing
cpp
tuple<VectorXd, VectorXd, VectorXd, VectorXd, double> 
spx_options_pricing_formula_cos_CTCL(...)

- Main pricing function for SPX options
- Uses Fourier cosine method
- Returns: Model volatilities, prices, and error metrics

### VIX Options Pricing
cpp
tuple<VectorXd, VectorXd, double> pricing_CTCM_JD(...)

- Pricing function for VIX options with jumps
- Returns: Volatilities, prices, and futures error

### Implied Volatility Calculation
cpp
double implied_volatility(double T, double K, double C, char tp)

- Computes implied volatility from option price
- Handles both call ('C') and put options

## Characteristic Functions

### Jump Process Characteristic Functions
cpp
VectorXcd char_L1(const VectorXd& para, const VectorXcd& u)
VectorXcd char_J1_neg(const VectorXd& para, const VectorXcd& u) 
VectorXcd char_J1_lnm(const VectorXcd& y, const VectorXd& para)

- Characteristic functions under different measures:
  - L1: Leverage-neutral measure
  - J1_neg: Negative jumps only
  - J1_lnm: Log-normal measure

### Composite Characteristic Functions
cpp
tuple<VectorXd, double, VectorXd> spot_Composite_JD(...)
tuple<VectorXd, double, VectorXd> spot_Composite_Heston(...)

- Combine jump and diffusion components
- Used for variance surface modeling

## Simulation Methods

### Variance Process Simulation
cpp
VectorXd get_conditional_V(...)
VectorXd get_X1(...)
VectorXd get_X2(...) 
VectorXd get_X3(...)

- Components for conditional variance simulation
- Used in Monte Carlo pricing

### Bessel Process Handling
cpp
VectorXd besseli_ratio(double nv, const VectorXd& z)
VectorXd sample_bessel(double nu, const VectorXd& z)

- Bessel function calculations and sampling
- Critical for jump process simulation

## Utility Functions

### Fourier Cosine Helpers
cpp
VectorXd calculate_chi(double a, double b, double c, double d, int N)
VectorXd calculate_psi(double a, double b, double c, double d, int N)

- Compute coefficients for Fourier cosine expansions
- Used in COS method option pricing

### Random Variate Generation
cpp
void calculate_x_simu(const VectorXd& rand, const VectorXd& x, const MatrixXd& F, VectorXd& x_simu_all)

- Inverse transform sampling implementation
- Generates simulated paths for pricing

## Error Handling & Calibration

### Calibration Functions
cpp
double error_function(...)
VectorXd error_function2(...)

- Calculate pricing errors for model calibration
- Support both absolute and relative error metrics
- Handle multiple asset classes (SPX, VIX)

## Dependencies

### Required Libraries
- **Eigen**: Linear algebra operations
- **Boost**: 
  - Mathematical distributions (normal, gamma, chi-squared)
  - Root-finding algorithms
  - Special functions
- **CSV2**: Data input/output
- **Chrono**: Performance timing

## Usage Examples

### Basic Option Pricing
cpp
// Set up parameters
VectorXd para(15); 
para << 1.5, 0.04, 0.3, 1.2, 0.035, 0.5, 0.2, 0.3, 1.5, 2.0, 0.7, 1.0, 1.2, 0.03, 0.4;

// Price SPX options
auto [vol_surf, prices, vol_VIX, VIX_prices, error] = 
    spx_options_pricing_formula_cos_CTCL(TK_S, TK_V, market_vol_SPX, market_vol_VIX, futures, para);


### Model Calibration
cpp
// Define error function for calibration
auto error_fn = const VectorXd& params {
    return error_function(params, TK_S, TK_V, market_vol_SPX, market_vol_VIX, futures, SPX_prices, VIX_prices);
};

// Run calibration
VectorXd optimal_params = calibrate(error_fn, initial_guess);


### Key Features
1. **High Performance**: Vectorized operations using Eigen
2. **Flexible Modeling**: Supports multiple stochastic processes
3. **Accurate Pricing**: Implements state-of-the-art Fourier methods
4. **Robust Calibration**: Comprehensive error metrics

## Implementation Notes

1. The code uses advanced C++ features including:
   - Template metaprogramming for performance
   - Lambda functions for flexible pricing
   - Modern memory management

2. Numerical stability is maintained through:
   - Careful handling of edge cases
   - Adaptive numerical methods
   - Precision-controlled algorithms

3. For production use:
   - Consider adding boundary checks
   - Implement additional error handling
   - Optimize hot code paths further

## File Structure

financial_math_lib/
├── src/
│   ├── models/           # Mathematical models
│   ├── numerical/        # Numerical methods
│   ├── pricing/          # Option pricing
│   ├── simulation/       # Monte Carlo methods
│   └── utils/           # Utility functions
├── include/             # Header files
├── tests/              # Test cases
└── examples/           # Usage examples


## Building the Library
bash
Clone the repository

git clone https://github.com/yourusername/financial_math_lib.git
cd financial_math_lib

Create build directory

mkdir build && cd build

Configure and build

cmake ..
make -j4

Run tests

make test


## Contributing
1. Fork the repository
2. Create a feature branch
3. Add tests for new functionality
4. Ensure all tests pass
5. Submit a pull request

## License
This project is licensed under the MIT License - see the LICENSE file for details.

## Citation
If you use this library in academic work, please cite:

@software{financial_math_lib,
  title = {Financial Mathematics Library},
  author = {Your Name},
  year = {2024},
  url = {https://github.com/yourusername/financial_math_lib}
}


## Support
For questions and support:
- Open an issue on GitHub
- Check the documentation in `/docs`
- Review example code in `/examples`

## Version History
- v1.0.0 (2024-01-01): Initial release with core functionality
- v0.9.0 (2023-12-15): Beta release with basic models

# Time-Changed Lévy Process Pricing Framework (Carr & Wu, 2004 Replication)

This repository provides a **high-performance Cython implementation** of the **Time-Changed Lévy Process (TCL)** option pricing framework proposed by **Carr & Wu (2004, Journal of Financial Economics)**.  
The model captures **jumps**, **stochastic volatility**, and **leverage effects** within a unified Lévy–time-change structure, allowing joint calibration to **SPX (equity)** and **VIX (variance)** derivatives.

---

## 📘 1. Theoretical Framework

### 1.1 Time-Changed Lévy Model
In the TCL setting, the log price process is given by:
\[
X_t = X_{T_t}, \quad T_t = \int_0^t v(s) ds
\]
where:
- \( X_t \) is a **Lévy process** (jump component)
- \( T_t \) is a **stochastic clock** (time change process)
- \( v(t) \) represents **instantaneous activity rate**

The stochastic time change induces **random volatility clustering**, while the Lévy driver controls **jump frequency and tail thickness**.

The characteristic function under the risk-neutral measure \(Q\) is:
\[
\Phi_X(u) = \mathbb{E}^Q[e^{i u X_t}] 
           = \mathbb{E}^Q[e^{-T_t \psi_X(u)}] 
           = L_{T_t}(\psi_X(u))
\]
where \( \psi_X(u) \) is the **Lévy characteristic exponent**, and \(L_{T_t}\) is the Laplace transform of the stochastic clock.

---

### 1.2 CGMY Lévy Subordinator

The Lévy driver \(X_t\) follows a **CGMY process**:
\[
\psi_X(u) = C \Gamma(-Y) \left[(M - i u)^Y - M^Y + (G + i u)^Y - G^Y \right]
\]
where:
- \(C\): overall jump intensity  
- \(G, M\): left/right tail parameters  
- \(Y\): jump activity index  

This allows both finite and infinite activity jumps.

---

### 1.3 Stochastic Clock — CIR Activity Rate

The time change process \(T_t\) is modeled by a **CIR process**:
\[
d v_t = \kappa (\theta - v_t) dt + \sigma \sqrt{v_t} dW_t
\]
\[
T_t = \int_0^t v_s ds
\]

The Laplace transform of \(T_t\) can be expressed in exponential–affine form:
\[
\mathbb{E}[e^{-\lambda T_t}] = e^{-A(t,\lambda) - B(t,\lambda)v_0}
\]
with \(A, B\) satisfying coupled **Riccati-type ODEs**.

---

## ⚙️ 2. Numerical Design

The implementation mirrors the affine transform method in Carr & Wu (2004) but extends it with **multi-factor structure** and **Cython-based numerical acceleration**.

### 2.1 Core ODE Solver — RK4 Integration

The ODE systems for \(A(t)\) and \(B(t)\):
\[
\begin{aligned}
\frac{dB}{dt} &= f(B; u, \text{params}) \\
\frac{dA}{dt} &= g(B; u, \text{params})
\end{aligned}
\]
are solved using a **4th-order Runge–Kutta (RK4)** scheme:

\[
y_{n+1} = y_n + \frac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4)
\]
where \(k_i\) are intermediate slopes evaluated at shifted points.  

This provides **high-order accuracy** and **numerical stability** for complex-valued affine systems.  
Three specialized RK4 versions are implemented:
- `rk4()` — general vectorized solver  
- `rk4_num()` — scalar complex-valued ODE solver  
- `rk4_T()` — solver for coupled time-change factors (multi-factor TCL)

---

### 2.2 Characteristic Function Construction

The **log-price characteristic function** is constructed as:
\[
\Phi(u; T) = \exp[A(T,u) + B(T,u) v_0]
\]
where:
- \(B(T,u)\) from `coef_B()` using RK4 on \(B'\)
- \(A(T,u)\) from cumulative integration of `derivative_A(B,u)`

Under the leverage-neutral measure, jump compensators are adjusted via:
\[
\phi(u) = -u \phi_1(-i) i + \phi_1(u)
\]

---

### 2.3 Fourier–COS Option Pricing

Option prices are evaluated using the **COS method** (Fang & Oosterlee, 2008):
\[
V_k = \frac{2}{A-L} \left[ \chi_k(L,A) - \psi_k(L,A) \right]
\]
\[
C(K) = e^{-rT} \sum_{k=0}^{N-1} \Re \left[ \Phi(u_k) V_k e^{i u_k \log(S_0/K)} \right]
\]

This is implemented in:
- `calculate_Vk_spx()`, `calculate_chi()`, `calculate_psi()`  
- `spx_options_pricing_formula_cos_CTCL()` — joint SPX/VIX pricing via Fourier expansion

---

### 2.4 Simulation of Variance and Time Change

Random time increments and conditional variances are generated using:
- **Non-central χ² distribution** for CIR terminal variance
- **Bessel and Gamma sampling** for conditional integrals
- **Inverse transform sampling** for stochastic time `U_t`

The total integrated variance \(V_T\) drives the VIX and the stochastic clock in the SPX process.

---

### 2.5 Multi-Factor Time-Change

A **second time-change factor** is included to capture long-memory or macro-volatility effects.  
The joint Laplace transform becomes:
\[
\Phi(u) = \exp[A_1 + B_1 v_t + A_2 + B_2 u_t]
\]
with each factor (primary and secondary) solved independently via RK4.

---

### 2.6 Calibration Module

The parameters are calibrated jointly to SPX and VIX smiles via nonlinear optimization (`scipy.optimize.minimize`) using the **mean squared implied volatility error**:

\[
\text{Loss} = \frac{1}{2}(\text{MSE}_{SPX} + \text{MSE}_{VIX})
\]
Calibration routines:
- `error_function()` — iterative Nelder–Mead optimization  
- `error_function2()` — final statistics output (RMSE, relative error, etc.)

---

## 🧮 3. Algorithm Flow Summary



# Financial Mathematics Library - Complete Mathematical Framework

## Mathematical Foundation

### Core Stochastic Processes

#### 1. CGMY Jump-Diffusion Process
The model combines a CGMY Lévy process with stochastic volatility:

**Characteristic Function:**

$$\phi_{CGMY}(u) = C\Gamma(-Y)\left[(M - iu)^Y - M^Y + (G + iu)^Y - G^Y\right]$$


**Negative Jumps Only:**

$$\phi_{CGMY}^{-}(u) = C\Gamma(-Y)G^Y\left[\left(1 + \frac{iu}{G}\right)^Y - 1\right]$$


#### 2. Heston Stochastic Volatility
The volatility follows a mean-reverting square-root process:

$$dv_t = \kappa(\theta - v_t)dt + \sigma\sqrt{v_t}dW_t^v$$


**Heston Characteristic Function:**

$$\phi_{Heston}(u) = \exp\left(A(T) + B(T)v_0\right)$$

where:

$$A(T) = \frac{\kappa\theta}{\sigma^2}\left[(\kappa - \delta)T - 2\log\left(1 - \frac{\delta - \kappa}{2\delta}(1 - e^{-\delta T})\right)\right]$$


$$B(T) = \frac{2iu(1 - e^{-\delta T})}{\delta + \kappa + (\delta - \kappa)e^{-\delta T}}$$


$$\delta = \sqrt{\kappa^2 - 2i u \sigma^2}$$


## Option Pricing Framework

### 1. Fourier Cosine Method (COS)

The COS method approximates the option price using Fourier cosine expansions:

**Call Option Price:**

$$C(t) \approx e^{-rT}\sum_{k=0}^{N-1} V_k \cdot \text{Re}\left[\phi\left(\frac{k\pi}{b-a}\right)e^{-ik\pi\frac{a}{b-a}}\right]$$


**Coefficients:**
- **V_k**: Fourier cosine coefficients for the payoff function
- **χ and ψ functions**: Trigonometric integrals for coefficient calculation

### 2. Characteristic Function Decomposition

The model decomposes the characteristic function into two factors:

**First Factor (Leverage-Neutral Measure):**
$$\phi_1(u) = \exp\left(A_1(T) + B_1(T)u_0\right)$$


**Second Factor (Volatility Process):**

$$\phi_2(u) = \exp\left(A_2(T) + B_2(T)u_{20}\right)$$


**Combined Characteristic Function:**

$$\phi(u) = \phi_1(u) \cdot \phi_2(u)$$


## Numerical Methods

### 1. Runge-Kutta ODE Solvers

**4th Order RK for ODEs:**

$$y_{n+1} = y_n + \frac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4)$$

where:

$$k_1 = hf(t_n, y_n)$$

math
k_2 = hf(t_n + \frac{h}{2}, y_n + \frac{k_1}{2})

math
k_3 = hf(t_n + \frac{h}{2}, y_n + \frac{k_2}{2})

math
k_4 = hf(t_n + h, y_n + k_3)


### 2. Monte Carlo Simulation

**Variance Process Simulation:**
math
V_T = X_1 + X_2 + X_3


**Component Distributions:**
- **X₁**: Poisson-weighted Gamma distribution
- **X₂**: Gamma distribution for stationary component
- **X₃**: Bessel process component

### 3. Bessel Process Sampling

**Bessel Ratio Approximation:**
math
\frac{I_{\nu+1}(z)}{I_\nu(z)} \approx \frac{z}{\nu + \frac{1}{2} + \sqrt{(\nu + \frac{3}{2})^2 + z^2}}


**Normal Approximation:**
math
X \sim \mathcal{N}(\mu, \sigma^2)

where:
math
\mu = \frac{z}{2}\frac{I_{\nu+1}(z)}{I_\nu(z)}

math
\sigma^2 = \frac{z^2}{4}\left(\frac{I_{\nu+2}(z)}{I_\nu(z)} - \left(\frac{I_{\nu+1}(z)}{I_\nu(z)}\right)^2\right) + \mu - \mu^2


## VIX Modeling

### VIX Formula Derivation

**VIX² Calculation:**
math
VIX^2 = \frac{2}{T}\sum_{i=1}^N \frac{\Delta K_i}{K_i^2}e^{rT}Q(K_i) - \frac{1}{T}\left(\frac{F}{K_0} - 1\right)^2


**Model Implementation:**
math
VIX = \sqrt{(a \cdot u_T + b \cdot v_T + c) + (a_2 \cdot u_{2T} + b_2 \cdot v_T + c_2)}


## Implied Volatility Calculation

### Black-Scholes Implied Volatility

**Call Option Formula:**
math
C = S_0N(d_1) - Ke^{-rT}N(d_2)

where:
math
d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}

math
d_2 = d_1 - \sigma\sqrt{T}


**Root Finding:**
math
f(\sigma) = BS(\sigma) - C_{\text{market}} = 0


## Error Metrics for Calibration

### 1. Mean Squared Error (MSE)
math
\text{MSE} = \frac{1}{N}\sum_{i=1}^N (\sigma_{\text{model}}^i - \sigma_{\text{market}}^i)^2


### 2. Absolute Error
math
\text{AbsError} = \frac{1}{N}\sum_{i=1}^N \sigma_{\text{model}}^i - \sigma_{\text{market}}^i



### 3. Relative Error
math
\text{RelError} = \sqrt{\frac{1}{N}\sum_{i=1}^N \left(\frac{\sigma_{\text{model}}^i - \sigma_{\text{market}}^i}{\sigma_{\text{market}}^i}\right)^2}


### 4. Composite Error Function
math
\text{TotalError} = w_1\cdot\text{MSE}_{SPX} + w_2\cdot\text{MSE}_{VIX} + w_3\cdot\text{FuturesError}


## Mathematical Components

### 1. Leverage-Neutral Measure Transformation

**Measure Change:**
math
\frac{d\mathbb{Q}^L}{d\mathbb{P}} = \exp\left(\int_0^T \lambda_s dW_s - \frac{1}{2}\int_0^T \lambda_s^2 ds\right)


**Characteristic Exponent Adjustment:**
math
\psi^L(u) = \psi(u) - iu\phi_1'(-i)


### 2. Fourier Inversion Techniques

**Damping Factor:**
math
\hat{C}(\alpha) = e^{\alpha k}C(k)


**Inversion Formula:**
math
C(k) = \frac{e^{-\alpha k}}{\pi}\int_0^\infty e^{-ivk}\hat{C}(v)dv


### 3. Numerical Integration Schemes

**Simpson's Rule:**
math
\int_a^b f(x)dx \approx \frac{h}{3}\left[f(x_0) + 4f(x_1) + 2f(x_2) + \cdots + f(x_n)\right]


**Adaptive Grid:**
- Fine grid near at-the-money options
- Coarser grid for deep in/out-of-the-money options

## Computational Optimizations

### 1. Vectorized Operations
- **Eigen library** for linear algebra
- **Broadcasting** for batch computations
- **Memory-efficient** matrix operations

### 2. Parallel Processing
- **Multi-threading** for Monte Carlo simulations
- **Parallel ODE solving** for different strikes/maturities

### 3. Caching and Memoization
- **Pre-computed** characteristic functions
- **Cached** numerical integrals
- **Reusable** intermediate results

## Model Parameters

### Core Parameters (15-dimensional):
1. **κ₁**: Mean reversion speed (1st factor)
2. **θ₁**: Long-term mean (1st factor)
3. **σ₁**: Volatility of vol (1st factor)
4. **κ₂**: Mean reversion speed (2nd factor)
5. **θ₂**: Long-term mean (2nd factor)
6. **σ₂**: Volatility of vol (2nd factor)
7. **C**: CGMY intensity parameter
8. **G**: CGMY decay parameter (right tail)
9. **M**: CGMY decay parameter (left tail)
10. **Y**: CGMY stability parameter
11. **ρ**: Leverage correlation
12. **u₀**: Initial value (1st factor)
13. **u₂₀**: Initial value (2nd factor)
14. **v₀**: Initial variance
15. **λ**: Jump intensity

This mathematical framework provides a comprehensive foundation for pricing SPX and VIX options using advanced stochastic processes and Fourier-based methods.

# Financial Mathematics Algorithm Documentation
Using **Eigen library** for SIMD-accelerated linear algebra:

cpp
// Vectorized characteristic function calculation
VectorXcd char_func = (A.array() + B.array() * v0.array()).exp();
## Mathematical Framework

### 1. Stochastic Volatility Model with Jumps

The model combines **Heston stochastic volatility** with **CGMY jump process**:

$$
dS_t = (r - q - \lambda m)S_tdt + \sqrt{V_t}S_tdW_t^S + S_tdJ_t
$$

$$
dV_t = \kappa(\theta - V_t)dt + \sigma\sqrt{V_t}dW_t^V
$$

Where:
- $J_t$ is a CGMY jump process
- $dW_t^SdW_t^V = \rho dt$
- $\lambda m$ is the jump compensator

### 2. Characteristic Function Framework

The core pricing methodology uses **characteristic functions** and **Fourier transforms**.

#### 2.1 CGMY Characteristic Function

For the CGMY process:

$$
\phi_{CGMY}(u) = C\Gamma(-Y)\left[(M - iu)^Y - M^Y + (G + iu)^Y - G^Y\right]
$$

**Negative jumps only version:**
$$
\phi_{CGMY}^{-}(u) = C\Gamma(-Y)G^Y\left[\left(1 + \frac{iu}{G}\right)^Y - 1\right]
$$

#### 2.2 Heston Characteristic Function

For the Heston model:

$$
\phi_{Heston}(u) = \exp\left(A(t,u) + B(t,u)V_0\right)
$$

Where:
$$
B(t,u) = \frac{2iu(1-e^{-dt})}{d + \kappa + (d-\kappa)e^{-dt}}
$$

$$
A(t,u) = \frac{\kappa\theta}{\sigma^2}\left[(\kappa-d)t - 2\log\left(\frac{1-ge^{-dt}}{1-g}\right)\right]
$$

With $d = \sqrt{\kappa^2 - 2\sigma^2iu}$ and $g = \frac{\kappa-d}{\kappa+d}$

### 3. Option Pricing Methodology

#### 3.1 Fourier-Cosine (COS) Method

The COS method expands the probability density function using cosine basis:

$$
f(x) \approx \sum_{k=0}^{N-1} A_k \cos\left(k\pi\frac{x-a}{b-a}\right)
$$

Where the coefficients are:
$$
A_k = \frac{2}{b-a} \Re\left\{\phi\left(\frac{k\pi}{b-a}\right)e^{-i\frac{k\pi a}{b-a}}\right\}
$$

**Option price formula:**
$$
C(t) \approx e^{-r\tau}\sum_{k=0}^{N-1} A_k V_k
$$

Where $V_k$ are the cosine coefficients for the payoff function.

#### 3.2 Runge-Kutta ODE Solving

For characteristic function coefficients, we solve Riccati equations using **4th-order Runge-Kutta**:

**General RK4 formulation:**
python
k1 = h * f(y_n)
k2 = h * f(y_n + k1/2)
k3 = h * f(y_n + k2/2)
k4 = h * f(y_n + k3)
y_{n+1} = y_n + (k1 + 2k2 + 2k3 + k4)/6


**Applied to characteristic function ODE:**
$$
\frac{dB}{dt} = -\kappa B + \psi(u) + \frac{\sigma^2}{2}B^2
$$

### 4. VIX Modeling Framework

#### 4.1 VIX Definition

$$
VIX_t^2 = \frac{1}{\tau}\mathbb{E}^\mathbb{Q}\left[\int_t^{t+\tau}V_sds\Big|\mathcal{F}_t\right] \times 100^2
$$

Where $\tau = 30$ days.

#### 4.2 Conditional Variance Simulation

The model simulates variance paths using **non-central chi-squared distributions**:

**Variance process conditional distribution:**
$$
V_T|V_t \sim \frac{\sigma^2(1-e^{-\kappa(T-t)})}{4\kappa}\chi_d^2\left(\lambda\right)
$$

Where:
- $d = \frac{4\kappa\theta}{\sigma^2}$ (degrees of freedom)
- $\lambda = \frac{4\kappa e^{-\kappa(T-t)}}{\sigma^2(1-e^{-\kappa(T-t)})}V_t$ (non-centrality)

### 5. Numerical Implementation Details

#### 5.1 Bessel Function Handling

For jump process modeling, we use **modified Bessel functions**:

**Bessel function ratio:**
$$
R_\nu(z) = \frac{I_{\nu+1}(z)}{I_\nu(z)}
$$

**Sampling from Bessel distribution** using normal approximation:
$$
X \sim \mu + \sigma Z,\quad Z \sim N(0,1)
$$

Where:
$$
\mu = \frac{z}{2}R_\nu(z),\quad \sigma^2 = \frac{z^2}{4}R_{\nu+1}(z)R_\nu(z) + \mu - \mu^2
$$

#### 5.2 Implied Volatility Calculation

Using **Brent's method** for root finding:

**Black-Scholes implied volatility solver:**
$$
f(\sigma) = BS(S,K,T,r,\sigma) - C_{market} = 0
$$

**For calls:**
$$
f(\sigma) = N(d_1) - KN(d_2) - C
$$

Where:
$$
d_1 = \frac{\ln(S/K) + (\sigma^2/2)T}{\sigma\sqrt{T}},\quad d_2 = d_1 - \sigma\sqrt{T}
$$

### 6. Monte Carlo Simulation Framework

#### 6.1 Variance Path Simulation

python
Step 1: Generate terminal variance

v_T = multiplier * ncx2.rvs(delta, lam, size=N_simu)

Step 2: Get conditional variance paths

V_T = get_conditional_V(quants, v0, v_T, T)

Step 3: Simulate jump components

u_T = get_U(para, V_T, h, N_simu)


#### 6.2 Inverse Transform Sampling

For distribution sampling:
python
def inverse_transform_sampling(rand, x, F):
    # Find index where F[i] < rand < F[i+1]
    index = bisect_left(F, rand) - 1
    # Linear interpolation
    return (xF[index+1]-rand + xrand-F[index])/(F[index+1]-F[index])


### 7. Model Calibration Framework

#### 7.1 Error Metric

**Weighted objective function:**
$$
L(\theta) = \frac{1}{2}MSE_{SPX} + \frac{1}{2}MSE_{VIX} + \lambda MSE_{Futures}
$$

Where:
$$
MSE = \frac{1}{n}\sum_{i=1}^n (IV_{model} - IV_{market})^2
$$



### 7. Key Mathematical Components

#### 7.1 Fourier Coefficients Calculation

**Chi function:**
$$
\chi(a,b,c,d) = \frac{\cos(k\pi\frac{d-a}{b-a})e^d - \cos(k\pi\frac{c-a}{b-a})e^c}{1 + (k\pi/(b-a))^2}
$$

**Psi function:**
$$
\psi(a,b,c,d) = \frac{b-a}{k\pi}\left[\sin\left(k\pi\frac{d-a}{b-a}\right) - \sin\left(k\pi\frac{c-a}{b-a}\right)\right]
$$

#### 7.2 Composite Spot-Jump Relationships

**Jump-diffusion composite:**
$$
VIX^2 = aU + bV + c
$$

Where coefficients are derived from characteristic functions:
$$
a = \frac{\phi_V(m_T\tau) - 1}{m_T\tau}\alpha
$$
$$
b = -\frac{\kappa\theta}{m_T}\phi\alpha
$$
$$
c = \alpha\left[\frac{\kappa\theta}{m_T^2\tau}(\phi_V-1) - \frac{\kappa\theta\kappa_V\theta_V}{m_Tm_V}(\phi-1)\right]
$$




In [ ]:
for i in range(len(vol_surf)):
    if vol_surf[i] == 0:
        if i == 0:
            vol_surf[i] = vol_surf[i+1]
        elif i == len(vol_surf)-1:
            vol_surf[i] = vol_surf[i-1]
        else:
            vol_surf[i] = (vol_surf[i-1] + vol_surf[i+1])/2

This mathematical framework provides a comprehensive foundation for pricing SPX and VIX options under a sophisticated stochastic volatility model with jumps, combining analytical methods with efficient numerical implementations.

In [1]:
lst = []

i = 2000
while i <= 3200:
    if i%7 == 0:
        if i%5 != 0: 
            lst.append(i)
    i = i + 1
        
print(lst)

[2002, 2009, 2016, 2023, 2037, 2044, 2051, 2058, 2072, 2079, 2086, 2093, 2107, 2114, 2121, 2128, 2142, 2149, 2156, 2163, 2177, 2184, 2191, 2198, 2212, 2219, 2226, 2233, 2247, 2254, 2261, 2268, 2282, 2289, 2296, 2303, 2317, 2324, 2331, 2338, 2352, 2359, 2366, 2373, 2387, 2394, 2401, 2408, 2422, 2429, 2436, 2443, 2457, 2464, 2471, 2478, 2492, 2499, 2506, 2513, 2527, 2534, 2541, 2548, 2562, 2569, 2576, 2583, 2597, 2604, 2611, 2618, 2632, 2639, 2646, 2653, 2667, 2674, 2681, 2688, 2702, 2709, 2716, 2723, 2737, 2744, 2751, 2758, 2772, 2779, 2786, 2793, 2807, 2814, 2821, 2828, 2842, 2849, 2856, 2863, 2877, 2884, 2891, 2898, 2912, 2919, 2926, 2933, 2947, 2954, 2961, 2968, 2982, 2989, 2996, 3003, 3017, 3024, 3031, 3038, 3052, 3059, 3066, 3073, 3087, 3094, 3101, 3108, 3122, 3129, 3136, 3143, 3157, 3164, 3171, 3178, 3192, 3199]


### 【Ans】1

In [1]:
[i for i in range(2000, 3201) if i%7 == 0 and i%5 != 0 ]

[2002,
 2009,
 2016,
 2023,
 2037,
 2044,
 2051,
 2058,
 2072,
 2079,
 2086,
 2093,
 2107,
 2114,
 2121,
 2128,
 2142,
 2149,
 2156,
 2163,
 2177,
 2184,
 2191,
 2198,
 2212,
 2219,
 2226,
 2233,
 2247,
 2254,
 2261,
 2268,
 2282,
 2289,
 2296,
 2303,
 2317,
 2324,
 2331,
 2338,
 2352,
 2359,
 2366,
 2373,
 2387,
 2394,
 2401,
 2408,
 2422,
 2429,
 2436,
 2443,
 2457,
 2464,
 2471,
 2478,
 2492,
 2499,
 2506,
 2513,
 2527,
 2534,
 2541,
 2548,
 2562,
 2569,
 2576,
 2583,
 2597,
 2604,
 2611,
 2618,
 2632,
 2639,
 2646,
 2653,
 2667,
 2674,
 2681,
 2688,
 2702,
 2709,
 2716,
 2723,
 2737,
 2744,
 2751,
 2758,
 2772,
 2779,
 2786,
 2793,
 2807,
 2814,
 2821,
 2828,
 2842,
 2849,
 2856,
 2863,
 2877,
 2884,
 2891,
 2898,
 2912,
 2919,
 2926,
 2933,
 2947,
 2954,
 2961,
 2968,
 2982,
 2989,
 2996,
 3003,
 3017,
 3024,
 3031,
 3038,
 3052,
 3059,
 3066,
 3073,
 3087,
 3094,
 3101,
 3108,
 3122,
 3129,
 3136,
 3143,
 3157,
 3164,
 3171,
 3178,
 3192,
 3199]

### 2. 计算从 0-9 中选出三个不重复数字能排成的三位数个数。打印所有排列出的三位数，并计算其中能被6整除的数据个数。

### 【Ans】2

In [4]:
numlist=[i for i in range(100,1000) if (str(i)[0] != str(i)[2])&(str(i)[0] != str(i)[1])&(str(i)[1] != str(i)[2])]
print(len(numlist),numlist)
listdiv6=[j for j in numlist if j%6 == 0]
print(len(listdiv6))

648 [102, 103, 104, 105, 106, 107, 108, 109, 120, 123, 124, 125, 126, 127, 128, 129, 130, 132, 134, 135, 136, 137, 138, 139, 140, 142, 143, 145, 146, 147, 148, 149, 150, 152, 153, 154, 156, 157, 158, 159, 160, 162, 163, 164, 165, 167, 168, 169, 170, 172, 173, 174, 175, 176, 178, 179, 180, 182, 183, 184, 185, 186, 187, 189, 190, 192, 193, 194, 195, 196, 197, 198, 201, 203, 204, 205, 206, 207, 208, 209, 210, 213, 214, 215, 216, 217, 218, 219, 230, 231, 234, 235, 236, 237, 238, 239, 240, 241, 243, 245, 246, 247, 248, 249, 250, 251, 253, 254, 256, 257, 258, 259, 260, 261, 263, 264, 265, 267, 268, 269, 270, 271, 273, 274, 275, 276, 278, 279, 280, 281, 283, 284, 285, 286, 287, 289, 290, 291, 293, 294, 295, 296, 297, 298, 301, 302, 304, 305, 306, 307, 308, 309, 310, 312, 314, 315, 316, 317, 318, 319, 320, 321, 324, 325, 326, 327, 328, 329, 340, 341, 342, 345, 346, 347, 348, 349, 350, 351, 352, 354, 356, 357, 358, 359, 360, 361, 362, 364, 365, 367, 368, 369, 370, 371, 372, 374, 375, 376, 378, 

### 3.构造一个函数，实现以下功能：返回斐波那契数列中第n项的值

### 【Ans】3

In [2]:
def func3(n):
    
    # special case n = 0,1,2
    if n <= 0:
        return None
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    
    # general case n >= 3
    else:
        a, b = 0, 1
        for i in range(3, n+1):
            a, b = b, a + b
            
    return b

### 4. 对于任意正序(从小到大）数组m和n，返回其合并后数组的中位数，不得使用外部模块。
#### E.g.: m = [1,2], n = [3,4] 
####          func4(m,n) =  (2+3)/2 = 2.5

### 【Ans】4

In [6]:
def func4(m,n):
    
    newlist = sorted(m + n)
    length = len(newlist)

    if length % 2 == 0:
        median = (newlist[length // 2 - 1] + newlist[length // 2]) / 2
    else:
        median = newlist[(length - 1) // 2]
   
    return median

In [7]:
m = [1,2]
n = [3,4] 
func4(m,n)


2.5

### 5. 定义函数func5，使其能够拟合OLS模型并根据参合系数返回拟数/残差/截距项。
#### 要求：
#### - 允许输入的x, y内含有nan。
#### - E.g.: x = [1,2,3,np.nan], y=[2,4,6,8]
#### - 除pandas, numpy 外不依赖任何其他外部模块

### 【Ans】5

In [4]:
import numpy as np
import pandas as pd

def func5(x, y, results='resid'):
    # Remove rows with NaN values

    mask = np.logical_and(~np.isnan(x), ~np.isnan(y))
    x_clean = x[mask]
    y_clean = y[mask]
    
    # Add intercept term 
    X = np.column_stack((np.ones(len(x_clean)), x_clean))
    
    # Calculate OLS coefficients
    beta = np.linalg.inv(X.T @ X) @ X.T @ y_clean
    
    # Calculate fitted values
    y_hat = X @ beta
    
    # Calculate residuals
    resid = y_clean - y_hat
    
    if results == 'coefficients':
        return beta
    elif results == 'fitted':
        return y_hat
    elif results == 'resid':
        return resid
    else:
        return None



### 6. 运行下面的python语句，close_price为股票日度收盘价模拟数据。
#### 6.1 试计算股票每日收益率
#### 6.2 过去3日收益率的移动波动率
#### 6.3 收益率和波动率(turnover)的截面相关系数。（第三问中缺失值以上一日结果填充）

In [32]:
import pandas as pd
import numpy as np

np.random.seed(0)

close_price = pd.DataFrame({
    "stock":['A', 'B', 'C', 'A', 'B','C','A','C','A','B','C','A','B','C','D','D','D','D','A','B','C','D'],
    "date":['20210104','20210104','20210104','20210102','20210102','20210102',
            '20210103', '20210103', '20210101','20210101','20210101',
            '20210105','20210105','20210105','20210102','20210103','20210104','20210105', 
            '20210106', '20210106', '20210106','20210106'],
    "close_price":np.random.normal(10,1,22)})

turnover = pd.DataFrame({
    "stock":['A'] * 6 + ['B'] * 6 + ['C'] * 6 + ['D'] * 6,
    "date":pd.date_range('20210101',periods=6, freq='D').to_list() * 4,
    "turnover":np.random.uniform(0,1,24)
})

In [33]:
close_price1 = close_price.groupby('stock').apply(lambda x: x.sort_values('date', ascending=True)).reset_index(drop=True)
close_price1['daily_return'] = close_price1.groupby('stock')['close_price'].pct_change()
print(close_price1)

   stock      date  close_price  daily_return
0      A  20210101     9.896781           NaN
1      A  20210102    12.240893      0.236856
2      A  20210103    10.950088     -0.105450
3      A  20210104    11.764052      0.074334
4      A  20210105    11.454274     -0.026333
5      A  20210106    10.313068     -0.099631
6      B  20210101    10.410599           NaN
7      B  20210102    11.867558      0.139950
8      B  20210104    10.400157     -0.123648
9      B  20210105    10.761038      0.034700
10     B  20210106     9.145904     -0.150091
11     C  20210101    10.144044           NaN
12     C  20210102     9.022722     -0.110540
13     C  20210103     9.848643      0.091538
14     C  20210104    10.978738      0.114746
15     C  20210105    10.121675     -0.078066
16     C  20210106     7.447010     -0.264251
17     D  20210102    10.443863           NaN
18     D  20210103    10.333674     -0.010551
19     D  20210104    11.494079      0.112294
20     D  20210105     9.794842   

C:\Users\reiyi\AppData\Local\Temp\ipykernel_8776\3783568819.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  close_price1 = close_price.groupby('stock').apply(lambda x: x.sort_values('date', ascending=True)).reset_index(drop=True)


In [40]:
close_price1['return_3_vol'] = np.array(close_price1.groupby('stock')['daily_return'].rolling(window=3).std())
print(close_price1)

   stock      date  close_price  daily_return  return_3_vol
0      A  20210101     9.896781           NaN           NaN
1      A  20210102    12.240893      0.236856           NaN
2      A  20210103    10.950088     -0.105450           NaN
3      A  20210104    11.764052      0.074334      0.171226
4      A  20210105    11.454274     -0.026333      0.090107
5      A  20210106    10.313068     -0.099631      0.087341
6      B  20210101    10.410599           NaN           NaN
7      B  20210102    11.867558      0.139950           NaN
8      B  20210104    10.400157     -0.123648           NaN
9      B  20210105    10.761038      0.034700      0.132687
10     B  20210106     9.145904     -0.150091      0.099934
11     C  20210101    10.144044           NaN           NaN
12     C  20210102     9.022722     -0.110540           NaN
13     C  20210103     9.848643      0.091538           NaN
14     C  20210104    10.978738      0.114746      0.123914
15     C  20210105    10.121675     -0.0

In [50]:
close_price1['date'] = pd.to_datetime(close_price1['date'])
close_price1['return_3_vol']=close_price1['return_3_vol'].fillna(method='ffill')
newdata = pd.merge(close_price1, turnover, on=['stock', 'date'], how='left')
newdata


C:\Users\reiyi\AppData\Local\Temp\ipykernel_8776\1225986315.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  close_price1['return_3_vol']=close_price1['return_3_vol'].fillna(method='ffill')


,stock,date,close_price,daily_return,return_3_vol,turnover
0,A,2021-01-01,9.896781,NaN,NaN,0.264556
1,A,2021-01-02,12.240893,0.236856,NaN,0.774234
2,A,2021-01-03,10.950088,-0.105450,NaN,0.456150
3,A,2021-01-04,11.764052,0.074334,0.171226,0.568434
4,A,2021-01-05,11.454274,-0.026333,0.090107,0.018790
5,A,2021-01-06,10.313068,-0.099631,0.087341,0.617635
6,B,2021-01-01,10.410599,NaN,0.087341,0.612096
7,B,2021-01-02,11.867558,0.139950,0.087341,0.616934
8,B,2021-01-04,10.400157,-0.123648,0.087341,0.681820
9,B,2021-01-05,10.761038,0.034700,0.132687,0.359508


In [51]:

correlation = newdata.groupby('date').apply(lambda x: x['turnover'].corr(x['return_3_vol']))
correlation


C:\Users\reiyi\AppData\Local\Temp\ipykernel_8776\1946992795.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  correlation = newdata.groupby('date').apply(lambda x: x['turnover'].corr(x['return_3_vol']))


date
2021-01-01    1.000000
2021-01-02   -0.061153
2021-01-03   -1.000000
2021-01-04   -0.911553
2021-01-05    0.757168
2021-01-06   -0.881390
dtype: float64

### 7.提供一个格式为pd.Series的价格序列，计算最大回撤和次大回撤（注意两者时间区间不能有重叠），以及两者的开始和结束时间【要求算法复杂度尽可能低，较低的复杂度有加分】

In [6]:
# 可以用给出的series来测试函数
np.random.seed(1000)
series = pd.Series(np.random.rand(300)+1,index = pd.date_range(end = '12/31/2018', periods = 300))
series.head()

2018-03-07    1.653590
2018-03-08    1.115007
2018-03-09    1.950283
2018-03-10    1.482191
2018-03-11    1.872475
Freq: D, dtype: float64

In [ ]:
import pandas as pd

def calculate_drawdown(series):
    max_drawdown = 0
    second_max_drawdown = 0
    max_drawdown_start = None
    max_drawdown_end = None
    second_max_drawdown_start = None
    second_max_drawdown_end = None
    
    max_price = series[0]
    
    for i in range(1, len(series)):
        price = series[i]
        
        if price > max_price:
            max_price = price
        
        drawdown = (max_price - price) / max_price * 100
        
        if drawdown > max_drawdown:
            second_max_drawdown = max_drawdown
            second_max_drawdown_start = max_drawdown_start
            second_max_drawdown_end = max_drawdown_end
            
            max_drawdown = drawdown
            max_drawdown_start = series.index[i-1]
            max_drawdown_end = series.index[i]
        
        elif drawdown > second_max_drawdown:
            second_max_drawdown = drawdown
            second_max_drawdown_start = series.index[i-1]
            second_max_drawdown_end = series.index[i]
    
    return max_drawdown, second_max_drawdown, max_drawdown_start, max_drawdown_end, second_max_drawdown_start, second_max_drawdown_end



In [21]:
import pandas as pd
import numpy as np
df1=pd.DataFrame(np.random.random(10))
df1.T.to_csv(r"C:\Users\Lenovo\Desktop\df1.csv",mode="a",header=False)
